### Important Imports

In [1]:
from abc import ABC, abstractmethod
from rlkeys import RedisKeys
import redis
import numpy as np
import random 
import math
import time
import os
import yaml
from robot_control import MoveToStart
from robot_control import GetMobileBasePosition
from robot_control import GetCartesianPosition
from robot_control import InchBase
from robot_control import SetMobileBasePosition
from robot_control import ResetRobot

### Step 1: Creating a folder to save model weights

In [2]:
weight_count = 0

if not os.path.isdir("./weights_folder"):
    os.makedirs("./weights_folder")
else:
    weight_count = len(os.listdir("./weights_folder"))

weight_name = "weights_{}.csv".format(weight_count)
WEIGHTS_PATH = "./weights_folder/{}".format(weight_name)   

print("Weights Path: {}".format(WEIGHTS_PATH))

Weights Path: ./weights_folder/weights_23.csv


#### Helper Function for generating Different exploration rates

This function is meant to come up with different exploration values for different subsections of movement during the training phase 

In [3]:
def GetExploreRates(epoch_num, e_min, e_max, decay, e_cutoffs):

    explore_rates = []

    for i in range(len(e_cutoffs)):
        if epoch_num >= e_cutoffs[i]:
            explore_rates.append(e_min + (e_max - e_min)*(math.exp(-1*decay*(epoch_num - e_cutoffs[i]))))
        else:
            explore_rates.append(e_max)

    return explore_rates

#### Step 1.5: Extract Neccessary positional data + Generate optimal path

In [4]:
from maze import FindShortestPath
from maze import CompressOptimalPath

data = {}

with open("mazecfg.yaml", "r") as file:
    data = yaml.safe_load(file)

grid = data['maze']
r_o = data['robotOrigin']

ROBOT_ORIGIN = np.array(r_o)

d_o = data['dogOrigin']
DOG_ORIGIN = np.array(d_o)

MAZE_POS = data['mazePos']
MAZE_SCALE = data['mazeScale']

optimalPath  = FindShortestPath(grid, (0,1))
compression = 15
optimalPath = CompressOptimalPath(optimalPath, compression)

START_POS = data['startPos']
END_POS = data['endPos']
GOAL_POS = np.array([0.501556,  0.308326])

print("Start Pos: {}".format(START_POS))
print("End Pos: {}".format(END_POS))
print("MAZE_POS: {}".format(MAZE_POS))
print("MAZE_SCALE: {}".format(MAZE_SCALE))
print("DOG_ORIGIN: {}".format(DOG_ORIGIN))
print("ROBOT_ORIGIN: {}".format(ROBOT_ORIGIN))

print("OPTIMAL PATH: {}".format(optimalPath))

Start Pos: [-0.3, -0.16, 0.6]
End Pos: [0.3, 0.16, 0.6]
MAZE_POS: [0, 0, 0.4]
MAZE_SCALE: [0.04, 0.04, 0.04]
DOG_ORIGIN: [ 0.9  -0.9   0.08]
ROBOT_ORIGIN: [-0.5 -0.5  0. ]
OPTIMAL PATH: [('down', 10), ('right', 2), ('up', 9), ('right', 6), ('down', 14)]


### Step 2: Define a reinforcement learning model

In [5]:
# from rlmodel import MazeModelContinuous

params = {"lr": 0.01,
          "episode_len": 500,
          "discount": 0.91,
          "lfreq": 1000}

redis_keys = RedisKeys()
redis_client = redis.Redis()

# model = MazeModelContinuous(redis_client, redis_keys, params, ROBOT_ORIGIN, GOAL_POS, MAZE_SCALE, num_states = 20, num_actions = 17)

### Step 3: Prepare Model for Training

In [6]:
# model.train_mode()

EPOCHS = 200
e_cutoffs = [0, 0, 25, 50, 50]
e_min = 0.2
e_max = 0.9
decay = 0.02
SAVE_INTERVAL = 1

### Train Simple Model: 12 Spots, 12 distances from the optimal point, 1 final goal with 200 reward

In [7]:
from rlmodel import MazeModelSimple
from robot_control import GetBaseAngle

GOAL_POS = np.array([0.425792 , -0.34814])
GOAL_POS_2 = np.array([0.425792 , 0.34814])

mm = MazeModelSimple(redis_client, redis_keys, params, ROBOT_ORIGIN, GOAL_POS_2, MAZE_SCALE, num_states = 144, num_actions = 3)

print(mm.Q_TABLE.shape)

(144, 3)


#### Load weights for simple model

In [53]:
mm.train_mode()
mm.load("./weights_folder/weights_14.csv")

In [8]:
epoch_start = 0

### Step 7: Train Simple Model

In [9]:
e_cutoffs = [0,0,0,0,0]

for epoch in range(epoch_start , EPOCHS):
    MoveToStart(redis_client, redis_keys, mm.r_o, START_POS)
    time.sleep(1)

    print("Started epoch: {}".format(epoch))

    explore_rates = GetExploreRates(epoch, e_min, e_max, decay, e_cutoffs)
    print("Explore Rate: {}".format(explore_rates[0]))
    
    mm.train(optimalPath, explore_rates[0], limit = 5)

    if epoch % SAVE_INTERVAL == 0:
        print("Saving Weights...")
        mm.save(WEIGHTS_PATH)
        print("Saved Weights at {}!".format(WEIGHTS_PATH))
    
    ResetRobot(redis_client, redis_keys)
    

Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Started epoch: 0
Explore Rate: 0.8999999999999999
0
Picked action 1
completed_action
Picked action 2
completed_action
Picked action 0
completed_action
Picked action 1
1
completed_action
Picked action 1
2
completed_action
Picked action 1
3
4
completed_action
Picked action 0
completed_action
Picked action 2
completed_action
Picked action 1
completed_action
Total Reward: -54
Episode Iters: 1477
Saving Weights...
Saved Weights at ./weights_folder/weights_23.csv!
Setting Base
Sett

ConnectionError: Error 61 connecting to localhost:6379. Connection refused.

In [54]:
mm.eval_mode(fast_eval=True)

In [55]:
MoveToStart(redis_client, redis_keys, mm.r_o, START_POS, fast = True)
time.sleep(3)
mm.eval(optimalPath, limit = 5)

0
Picked action 0
completed_action
Picked action 0
completed_action
Picked action 2
completed_action
Picked action 0
completed_action
Picked action 0
completed_action
Picked action 0
completed_action
Picked action 0
completed_action
Picked action 0
completed_action
Picked action 0
completed_action
Picked action 0
completed_action
Picked action 0
completed_action
Picked action 0
completed_action
Picked action 0
completed_action
Picked action 0
completed_action
Picked action 0
completed_action
Picked action 0
completed_action
Picked action 0
completed_action
Picked action 0
completed_action
Picked action 0
completed_action
Picked action 0
completed_action
Picked action 0
completed_action
Picked action 0
completed_action
Picked action 0
completed_action
Picked action 0
completed_action
Picked action 0
completed_action
Picked action 0
completed_action
Picked action 0
completed_action
Picked action 0
completed_action
Picked action 1
1
completed_action
Picked action 2
completed_action
Picked

#### Auxiliary Functions

In [1]:
mm.performAction(0)

NameError: name 'mm' is not defined

In [ ]:
mm.performAction(1)

In [ ]:
mm.performAction(2)

In [10]:
MoveToStart(redis_client, redis_keys, mm.r_o, START_POS, fast = True)

In [22]:
mm.SolveMaze(optimalPath)
mm.path_index += 1

4


In [ ]:
print(GetBaseAngle(mm.redis_client, mm.redis_keys, ROBOT_ORIGIN))
pos = GetMobileBasePosition(mm.redis_client, mm.redis_keys, ROBOT_ORIGIN)
print(pos)

print(np.linalg.norm(pos - GOAL_POS))
print(GOAL_POS)

### Sanity Check

In [ ]:
combos = [12,6,2]
num_map = {}
min_num = 1000
max_num = -1


for i in range(combos[0]):
    for j in range(combos[1]):
        for k in range(combos[2]):

            state_num = mm.toStateNum([i,j,k])
            min_num = min(min_num, state_num)
            max_num = max(max_num, state_num)
            if state_num not in num_map:
                num_map[state_num] = 0
            else:
                raise Exception("Duplicate found")
            
print(min_num , max_num)

### This is what is required to have fine grain control over robot movement

In [20]:
#Let us test out mm's inch base

InchBase(mm.redis_client, mm.redis_keys, mm.r_o, increment=0.05, direction = 1)

array([-0.54239178,  0.09116554])

In [22]:
#Let us test out mm.performAction

target_angle, direction = mm.performAction(1)
print(target_angle*180/math.pi)
print(direction)
print(360/12)

print(GetBaseAngle(mm.redis_client, mm.redis_keys, mm.r_o)* 180/math.pi)

344.99999999999994
-1
30.0
9.908571277149585


In [24]:
#perform action is working as intended

print(3.94* 180/math.pi)
print(3.49 * 180/math.pi)

225.74537128154438
199.96227050065733


In [18]:
actions = [2,1,2,1,2,2, 1,1]
base_state = "start_action"
total_actions = 0



while total_actions < len(actions):
    base_angle = GetBaseAngle(mm.redis_client, mm.redis_keys, mm.r_o)

    if base_state == "start_action":
        a = actions[total_actions]
        print("Picked action {}".format(a))
        total_actions += 1
        target_angle, dir = mm.performAction(a)
        base_state = "start_submovement"
        print("curr_base_angle = {}, target angle = {}, direction = {}".format(base_angle* 180/math.pi, target_angle*180/math.pi, dir))

    elif base_state == "start_submovement":
        if np.linalg.norm(base_angle - target_angle) < 0.03 or a == 0:
            base_state = "completed_action"
            continue
        target_base_pos = InchBase(mm.redis_client, mm.redis_keys, mm.r_o, mm.optimal_r, mm.a_increment, direction)
        base_state = "in_submovement"
        sub_base_iter = 0

    elif base_state == "in_submovement":
        base_pos = GetMobileBasePosition(mm.redis_client, mm.redis_keys, mm.r_o)

        sub_base_iter += 1

        if np.linalg.norm(base_pos - target_base_pos) < 1e-2 or sub_base_iter > 10:
            base_state = "start_submovement"

    elif base_state == "completed_action":
        base_state = "start_action"

Picked action 2
curr_base_angle = 208.07253595896933, target angle = 225.0, direction = 1


NameError: name 'direction' is not defined

In [57]:
actions = [1,1,1,1,1,2,2,2,2,2,1]
base_state = "start_action"
total_actions = 0

MoveToStart(redis_client, redis_keys, mm.r_o, START_POS, fast = True)


while total_actions < len(actions):
    time.sleep(1/1000)
    base_pos = GetMobileBasePosition(mm.redis_client, mm.redis_keys, mm.r_o)

    if base_state == "start_action":
        a = actions[total_actions]
        total_actions += 1
        final_target, dir = mm.performAction(a)
        base_state = "start_submovement"

    elif base_state == "start_submovement":
        if np.linalg.norm(final_target - base_pos) < 0.1 or a == 0:
            print("completed_action")
            base_state = "completed_action"
            continue
        target_base_pos = InchBase(mm.redis_client, mm.redis_keys, mm.r_o, mm.optimal_r, 0.3, dir)
        base_state = "in_submovement"
        sub_base_iter = 0

    elif base_state == "in_submovement":
        sub_base_iter += 1

        if np.linalg.norm(base_pos - target_base_pos) < 1e-2 or sub_base_iter > 100:
            base_state = "start_submovement"

    elif base_state == "completed_action":
        base_state = "start_action"

completed_action
completed_action
completed_action
completed_action
completed_action
completed_action
completed_action
completed_action
completed_action
completed_action


In [21]:
MoveToStart(redis_client, redis_keys, mm.r_o, START_POS, fast = True)
ResetRobot(redis_client, redis_keys)


In [17]:
import json

redis_client.set(redis_keys.reset, json.dumps(0))

True

In [13]:
MoveToStart(redis_client, redis_keys, mm.r_o, START_POS)

Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base
Setting Base


In [15]:
for i in range(20):
    print(i)
    ResetRobot(redis_client, redis_keys)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


ConnectionError: Error 61 connecting to localhost:6379. Connection refused.